In [ ]:
!pip install torch numpy scikit-learn iterative-stratification

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

In [ ]:
file_path = "/content/drive/MyDrive/diseasedataset/properfinal.csv"
df = pd.read_csv(file_path)

In [ ]:
# One-hot encode diseases column
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(df[["diseases"]])

# split into features & labels
X = df.drop(columns=["diseases"]).values
y = y_encoded

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210623 entries, 0 to 210622
Columns: 379 entries, Unnamed: 0 to diseases
dtypes: int64(378), object(1)
memory usage: 609.0+ MB


In [ ]:
mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# splitting into train-test (one iteration only)
for train_index, test_index in mskf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    break  # only take the first split

In [ ]:
class HDCClassifier:
    def __init__(self, dim=1000):
        self.dim = dim  # Hyperdimensional vector size
        self.prototypes = {}  # Stores prototype vectors per class

    def _encode(self, x):
        """ Convert input features into hyperdimensional binary vectors. """
        return torch.sign(torch.tensor(x, dtype=torch.float32))  # Convert to float

    def fit(self, X_train, y_train):
        """ Train by averaging class-specific hyperdimensional vectors. """
        for label in np.unique(y_train):
            class_vectors = [self._encode(X_train[i]) for i in range(len(y_train)) if y_train[i] == label]
            self.prototypes[label] = torch.mean(torch.stack(class_vectors), dim=0).float()  # Ensure float type

    def predict(self, X_test):
        """ Predict based on similarity to stored class prototypes. """
        predictions = []
        for x in X_test:
            encoded_x = self._encode(x)
            similarities = {label: torch.dot(encoded_x, proto) for label, proto in self.prototypes.items()}
            predictions.append(max(similarities, key=similarities.get))  # Closest class
        return np.array(predictions)

In [ ]:
y_train_labels = y_train.argmax(axis=1)  # Convert from one-hot to single-label vector
y_test_labels = y_test.argmax(axis=1)

In [ ]:
# Initialize & train
model = HDCClassifier()
model.fit(X_train, y_train_labels)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
acc = accuracy_score(y_test_labels, y_pred)
print(acc)

0.8618364827651696


In [ ]:
precision = precision_score(y_test_labels, y_pred, average='macro', zero_division=1)
print("Precision:", precision)

Precision: 0.8653660324290485


In [ ]:
recall = recall_score(y_test_labels, y_pred, average='macro', zero_division=1)
print("Recall:", recall)

Recall: 0.8687788396700531


In [ ]:
clssf = classification_report(y_test_labels, y_pred)
print(clssf)

              precision    recall  f1-score   support

           0       0.92      0.74      0.82       228
           1       0.98      0.91      0.94       302
           2       0.86      0.65      0.74       311
           3       0.64      0.76      0.69       228
           4       0.94      0.96      0.95       232
           5       0.73      0.84      0.78       170
           6       0.99      0.87      0.93       303
           7       0.86      0.90      0.88       223
           8       0.92      0.91      0.91       228
           9       0.60      0.94      0.73       127
          10       0.88      0.92      0.90       128
          11       0.94      0.98      0.96       167
          12       0.91      0.97      0.94       227
          13       0.97      0.92      0.94       224
          14       0.91      0.89      0.90       150
          15       0.94      0.93      0.93       299
          16       0.83      0.95      0.88       233
          17       0.97    